<a href="https://colab.research.google.com/github/barauna-lo/CAP4213-Deep-Learning/blob/main/PyTorch_MLP_Aprendizado_Profundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src=https://raw.githubusercontent.com/barauna-lo/CAP4213-Deep-Learning/main/logoinpe.png>


# CAP-421-3: Aprendizado Profundo

## Atividade 2: Exemplo de aplicação em Python

* Luan Orion Baraúna 

* Renato Maximiniano 

* Vinicius Monego

# Introdução

A aplicação deste trabalho é em previsão do tempo, mais especificamente previsão de precipitação, utilizando redes neurais artificiais do tipo MLP (Multi Layer Perceptron) como alternativa à previsão numérica do tempo (NWP). A NWP possui um custo computacional muito alto, e o aprendizado profundo apresenta a vantagem de execução mais eficiente em comparação à NWP.

Este trabalho foi implementado e apresentado pelo aluno [Vinicius Monego](http://lattes.cnpq.br/1175145401785658) na disciplina de Inteligência Artificial da CAP/INPE em 2021. A implementação da rede neural foi realizada no framework [PyTorch](https://pytorch.org/) puro, sem uso de interfaces.

## Importando as Bibliotecas

<img src="https://upload.wikimedia.org/wikipedia/commons/9/96/Pytorch_logo.png" width = 300 >


<img src='https://pandas.pydata.org/static/img/pandas_white.svg  ' width = 300>

<img src='https://user-images.githubusercontent.com/1217238/65354639-dd928f80-dba4-11e9-833b-bc3e8c6a737d.png' width = 300>

<img src='https://scipy-lectures.org/_images/scikit-learn-logo.png' width =300>

<img src='https://matplotlib.org/stable/_static/logo2_compressed.svg' width =300>




In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as nnf
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Garantir a reprodutibilidade da rede

torch.manual_seed(42)
torch.use_deterministic_algorithms(False)

Os dados utilizados são do [GPCP](https://psl.noaa.gov/data/gridded/data.gpcp.html) (Global Precipitation Climate Project), que contém diferentes tipos de dados climáticos (vento, precipitação, humidade, geopotencial, temperatura e pressão) coletados mensalmente de 1979 a 2020 em diferentes pontos de grade, identificados pelas coordenadas de latitude e longitude. O objetivo da rede é aprender com estes dados para estimar o nível acumulado de precipitação do mês seguinte.

Os dados foram convertidos do formato original (NetCDF) para uma panilha XLSX para facilitar a manipulação. A região extraída dos dados é a América do Sul.

<centet> <img src="https://psl.noaa.gov/data/gridded/images/small/gpcp.png" alt="centered image" width = 600> </centet>

In [ ]:
weather_data = pd.read_excel('/content/drive/My Drive/dados-outono.xlsx')

Os dados como estão na planilha não estão normalizados. Os Eles precisam ser normalizados porque as funções de ativação dependem do dado normalizado para introduzir a não-linearidade na rede.

Para normalização, é utilizado o pacote `scikit-learn` e seu recurso de normalização do tipo Min-Max.

In [ ]:
x_scaler = MinMaxScaler()
x_scaler.fit(weather_data.loc[:, 'v850':'prec_GPCP'])

y_scaler = MinMaxScaler()
y_scaler.fit(weather_data.loc[:, 'prec_GPCP'].to_numpy().reshape(-1, 1))

MinMaxScaler(copy=True, feature_range=(0, 1))

O treinamento de um modelo de machine learning normalmente possui três tipos de conjuntos de dados:

- **Treinamento**: Parte dos dados que influenciam no diretamente no ajuste dos pesos. 
- **Validação**: Parte dos dados da qual se calculam as métricas para saber se a rede está aprendendo. Faz parte do treinamento, mas não influencia no ajuste dos pesos.
- **Teste**: Não faz parte do conjunto de treinamento. É a parte dos dados que a rede não viu durante o treinamento e serve para avaliar a rede já treinada.

O conjunto de treinamento+validação é escolhido como sendo abaixo de 2017, enquanto o de teste de 2017 e após.

In [ ]:
trainval_data = weather_data.loc[weather_data['year'] < 2017]
test_data = weather_data.loc[weather_data['year'] >= 2017]

A classe que prepara os dados (leitura e processamento dos dados, normalização e seleção das *features*) é vista abaixo.

In [ ]:
class GPCP(Dataset):

  def __init__(self, data, transform=None):
    """Read the dataset and assign varbiales"""
    
    self._data = data
    self.x, self.y = self._prepare_data(self._data)

    self.x = torch.from_numpy(self.x).float().to(device)
    self.y = torch.from_numpy(np.expand_dims(self.y, axis=1)).float().to(device)
    self._n_samples = self.x.shape[0]

    self._transform = transform

  def _prepare_data(self, data):
    """Sort data by latitude/longitude, select features"""
    data_group = [pd.DataFrame(y) for x, y in data.groupby(['lat', 'lon'], as_index=False)]

    x = []
    y = []

    for df in data_group:
      x.append(df.loc[:, 'v850':'prec_GPCP'][:-1].to_numpy())
      y.append(df.loc[:, 'prec_GPCP'][1:].to_numpy())

    x = np.array(x)
    xshp = x.shape
    x = x.reshape((xshp[0]*xshp[1], xshp[2]))

    y = np.array(y)
    yshp = y.shape
    y = y.reshape((yshp[0] * yshp[1]))

    return x, y

  def __getitem__(self, index):
    """Index the dataset"""
    sample = self.x[index], self.y[index]

    if self._transform:
      sample = self._transform(sample)

    return sample

  def __len__(self):
    """Return len()"""
    return self._n_samples

E a transformação do PyTorch que normaliza os dados e é passada como parâmetro para a classe acima é vista abaixo.

In [ ]:
class NormalizeTransform:
  """Normalize the dataset"""
  def __init__(self, x_scaler, y_scaler):
    self._x_scaler = x_scaler
    self._y_scaler = y_scaler

  def __call__(self, sample):
    sample_x = np.expand_dims(sample[0].cpu(), axis=0)
    sample_y = sample[1].cpu().reshape(-1, 1)
    x_normalized = self._x_scaler.transform(sample_x)
    y_normalized = self._y_scaler.transform(sample_y)
    x_normalized = torch.from_numpy(x_normalized).float().to(device)
    y_normalized = torch.from_numpy(y_normalized).float().to(device)
    return x_normalized, y_normalized

Na próxima célula ocorre a preparação (seleção das *features* e normalização) do conjunto de teste.

In [ ]:
dataset = GPCP(trainval_data, transform=NormalizeTransform(x_scaler, y_scaler))

test_data_input = test_data.loc[:, 'v850':'prec_GPCP']
test_dataset = y_scaler.transform(test_data_input)
# test_dataset = test_dataset.to_numpy()
test_dataset = torch.from_numpy(test_dataset)

Na próxima célula ocorre a separação do conjunto de treinamento entre os dados e treinamento e os dados de validação (80% e 20%, respectivamente).

In [ ]:
# split the dataset into train, validation and test sets

total_count = len(dataset)

train_count = int(0.8 * total_count)
valid_count = int(0.2 * total_count)

train_dataset, valid_dataset = torch.utils.data.random_split(
    dataset, (train_count, valid_count))

Na próxima célula ocorre o carregamento dos dados de treinamento para um tipo de dado que é usado pelo PyTorch para treinar a rede (DataLoader).

In [ ]:
# load the dataset

batch_size = 8

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)

Na próxima célula ocorre a definição da arquitetura da rede neural. É uma rede de 2 camadas escondidas, com 30 e 35 neurônios respectivamente, utilizando a função ReLU parametrizada como função de ativação e a função Sigmoide na camada de saída.

In [ ]:
class MLP(nn.Module):

  def __init__(self, input_size):
    super(MLP, self).__init__()
    self.layers = nn.Sequential(
      nn.Linear(input_size, 30),
      nn.PReLU(),
      nn.Linear(30, 35),
      nn.PReLU(),
      nn.Linear(35, 1),
      nn.Sigmoid()
    )

  def forward(self, x):
    return self.layers(x)

Após, o modelo é criado utilizando a função de erro médio quadrático como função custo e o otimizador Adadelta como função de otimização. Também é escolhido o número de épocas.

In [ ]:
model = MLP(9).to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adadelta(model.parameters())
num_epochs = 1000

A próxima célula é o treinamento da rede. Ao final, o objeto `model` será a rede treinada e poderá ser usada para avaliar o conjunto de teste.

In [ ]:
# train stage

print(f"Device: {device}")

train_losses = []
valid_losses = []

for epoch in range(num_epochs):
  train_loss = 0.0
  valid_loss = 0.0
  for i, (x, y) in enumerate(train_dataloader):
    model.train()

    # forward pass
    output = model(x)
    loss = loss_function(output, y)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item() * x.size(0)

  with torch.no_grad():
    for i, (x, y) in enumerate(valid_dataloader):
      model.eval()

      output = model(x)
      loss = (loss_function(output, y))

      valid_loss += loss.item() * x.size(0)

  train_loss /= len(train_dataset)
  valid_loss /= len(valid_dataset)

  print(f"Epoch: {epoch+1}. Train loss: {train_loss:.8f}. Validation loss: {valid_loss:.8f}")

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

train_losses = np.array(train_losses)
valid_losses = np.array(valid_losses)

Device: cuda
Epoch: 1. Train loss: 0.00803209. Validation loss: 0.00553512
Epoch: 2. Train loss: 0.00563681. Validation loss: 0.00515575
Epoch: 3. Train loss: 0.00545424. Validation loss: 0.00519990
Epoch: 4. Train loss: 0.00535168. Validation loss: 0.00548680
Epoch: 5. Train loss: 0.00531450. Validation loss: 0.00496251
Epoch: 6. Train loss: 0.00525262. Validation loss: 0.00492629
Epoch: 7. Train loss: 0.00521796. Validation loss: 0.00538865
Epoch: 8. Train loss: 0.00518553. Validation loss: 0.00507075
Epoch: 9. Train loss: 0.00517002. Validation loss: 0.00496628
Epoch: 10. Train loss: 0.00514838. Validation loss: 0.00498768
Epoch: 11. Train loss: 0.00514907. Validation loss: 0.00497278
Epoch: 12. Train loss: 0.00512823. Validation loss: 0.00517916
Epoch: 13. Train loss: 0.00513116. Validation loss: 0.00494326
Epoch: 14. Train loss: 0.00511223. Validation loss: 0.00492396
Epoch: 15. Train loss: 0.00511568. Validation loss: 0.00491631
Epoch: 16. Train loss: 0.00507333. Validation loss:

Como visto acima, o erro obtido á da ordem de $10^{-3}$.

A curva de aprendizado pode ser vista na saída da célula abaixo. Esta curva tem como eixos as épocas e o valor da função custo na época. É importante para ver a evolução do treinamento e identificar se a rede está passando por *overfitting* ou se o treinamento estagnou, e em qual época isso ocorre.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=[10, 7])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(train_losses, label="Train loss")
plt.plot(valid_losses, label="Validation loss")
plt.legend()

NameError: ignored

Para finalizar, o erro pode ser calculado também no conjunto de teste e a rede pode ser salva num arquivo .pth.

In [ ]:
# test the network

out_array = []

with torch.no_grad():
  test_loss = []
  for x in test_dataset:
    output = model(x)
    out_array.append(output.numpy())

In [ ]:
torch.save(model.state_dict(), "/content/drive/My Drive/mlp-optimized.pth")